In [1]:
!docker run -p 9000:9000 -p 9001:9001 \
  -e MINIO_ROOT_USER=minio \
  -e MINIO_ROOT_PASSWORD=minio123 \
  -v $(pwd)/minio_data:/data \
  minio/minio server /data --console-address ":9001"

INFO: Formatting 1st pool, 1 set(s), 1 drives per set.
INFO: WARNING: Host local has more than 0 drives of set. A host failure will result in data becoming unavailable.
MinIO Object Storage Server
Copyright: 2015-2025 MinIO, Inc.
License: GNU AGPLv3 - https://www.gnu.org/licenses/agpl-3.0.html
Version: RELEASE.2025-09-07T16-13-09Z (go1.24.6 linux/arm64)

API: http://172.17.0.2:9000  http://127.0.0.1:9000 
WebUI: http://172.17.0.2:9001 http://127.0.0.1:9001 

Docs: https://docs.min.io
^C


In [1]:
!pip install minio

from minio import Minio
from pathlib import Path


MINIO_ENDPOINT = "127.0.0.1:9000"   
MINIO_USER     = "minio"
MINIO_PASS     = "minio123"
BUCKET         = "model-registry"

client = Minio(MINIO_ENDPOINT, access_key=MINIO_USER, secret_key=MINIO_PASS, secure=False)

if not client.bucket_exists(BUCKET):
    client.make_bucket(BUCKET)

root = Path("model_registry")  
uploaded = []
for p in root.rglob("*"):
    if p.is_file():
        obj = p.relative_to(root).as_posix()   
        client.fput_object(BUCKET, obj, str(p))
        uploaded.append(obj)

print(f"Uploaded {len(uploaded)} objects.")
uploaded[:10]  # show a few


Uploaded 8 objects.


['model1/model.pt',
 'model1/model.joblib',
 'model3/model.pt',
 'model3/model.joblib',
 'model4/model.pt',
 'model4/model.joblib',
 'model2/model.pt',
 'model2/model.joblib']

In [7]:
!cd server
!docker build -t id_card:latest .

[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.0s (2/2) FINISHED                           docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 2B                                         0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
ERROR: failed to solve: failed to read dockerfile: open /var/lib/docker/tmp/buildkit-mount2186791928/Dockerfile: no such file or directory


In [ ]:
!cd router
!docker build -t router:latest .

In [8]:
!docker run -d --name model2_a -p 50051:50051 \
  -e VARIANT=model2 \
  -e MINIO_ENDPOINT=host.docker.internal:9000 \
  -e MINIO_USER=minio -e MINIO_PASS=minio123 \
  -e REGISTRY_BUCKET=model-registry \
  id_card:latest


!docker run -d --name model4_a -p 50052:50051 \
  -e VARIANT=model4 \
  -e MINIO_ENDPOINT=host.docker.internal:9000 \
  -e MINIO_USER=minio -e MINIO_PASS=minio123 \
  -e REGISTRY_BUCKET=model-registry \
  id_card:latest

!docker run -d --name model2_b -p 50061:50051 \
  -e VARIANT=model2 \
  -e MINIO_ENDPOINT=host.docker.internal:9000 \
  -e MINIO_USER=minio -e MINIO_PASS=minio123 \
  -e REGISTRY_BUCKET=model-registry \
  id_card:latest


ffed758b1f9f939fa064e2e7e3508ec97f0267984c3bb7b63413bfa2c192085b
291cc6cf4898dc5459a02fe971494e7e248732e0036799e40597f2138e861b53
16107340522b329b7e9441d92e76a6f2f052e04265c7225e90d7b3aceead05e6


In [14]:
!docker run -d --name router -p 50050:50050 \
  -e BACKENDS="model2=host.docker.internal:50051;host.docker.internal:50061,model4=host.docker.internal:50052" \
  router:latest


d64c99c62adb135d14ba7bc84d21cf487b72a4b0f2ff353dfe1cdb78a96bed80


In [20]:
import grpc
import cv2
import numpy as np

# ensure we can import the stubs
import sys
sys.path.append("server")

import inference_pb2 as pb
import inference_pb2_grpc as pbg

ROUTER_ADDR = "localhost:50050"

# open GRPC channel to router
channel = grpc.insecure_channel(ROUTER_ADDR)
stub = pbg.InferenceStub(channel)

def predict_image(img_path):
    # read the image
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {img_path}")

    # encode to jpeg bytes
    ok, enc = cv2.imencode(".jpg", img)
    img_bytes = enc.tobytes()

    # prepare request
    req = pb.PredictRequest(image=img_bytes)

    # call using future to capture metadata (which worker served)
    call = stub.Predict.future(req)
    resp = call.result()
    trailers = {k: v for k, v in call.trailing_metadata()}

    print("Request:", img_path)
    print("Prediction:", resp.y)
    print(f"Inference time: {resp.infer_ms:.2f} ms")
    print("Served by model:", trailers.get("x-variant"))
    print("Backend endpoint:", trailers.get("x-target"))

    return resp


In [21]:
predict_image("sample.jpg")

Request: sample.jpg
Prediction: 32.0
Inference time: 46.70 ms
Served by model: model2
Backend endpoint: host.docker.internal:50051


y: 32
infer_ms: 46.696749999682652

In [22]:
predict_image("sample.jpg")

Request: sample.jpg
Prediction: 32.0
Inference time: 38.68 ms
Served by model: model4
Backend endpoint: host.docker.internal:50052


y: 32
infer_ms: 38.684750001266366

As we can see above that it is using different model which is selected randomly at this point. 